### MLflow Project and Model Example

<table>
  <tr><td>
    <img src="https://raw.githubusercontent.com/dmatrix/mlflow-workshop-part-2/master/images/models_1.png"
         alt="Bank Note " width="400">
  </td></tr>
  <tr><td> Save a Keras Model Flavor and load as both Keras Native Flavor and PyFunc Flavor</td></tr>
  <tr><td>
    <img src="https://raw.githubusercontent.com/dmatrix/mlflow-workshop-part-2/master/images/models_2.png"
         alt="Bank Note " width="400">
  </td></tr>
</table>

In [0]:
%pip install keras

Python interpreter will be restarted.
Collecting keras
 Downloading Keras-2.4.3-py2.py3-none-any.whl (36 kB)
Requirement already satisfied: pyyaml in /local_disk0/.ephemeral_nfs/envs/pythonEnv-2efa82e9-eb2f-408c-a2c5-719a27e50a22/lib/python3.7/site-packages (from keras) (5.3.1)
Requirement already satisfied: h5py in /local_disk0/.ephemeral_nfs/envs/pythonEnv-2efa82e9-eb2f-408c-a2c5-719a27e50a22/lib/python3.7/site-packages (from keras) (2.10.0)
Requirement already satisfied: scipy>=0.14 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-2efa82e9-eb2f-408c-a2c5-719a27e50a22/lib/python3.7/site-packages (from keras) (1.4.1)
Requirement already satisfied: numpy>=1.9.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-2efa82e9-eb2f-408c-a2c5-719a27e50a22/lib/python3.7/site-packages (from keras) (1.18.1)
Requirement already satisfied: six in /local_disk0/.ephemeral_nfs/envs/pythonEnv-2efa82e9-eb2f-408c-a2c5-719a27e50a22/lib/python3.7/site-packages (from h5py->keras) (1.14.0)
Installing collected packages: keras
Successfully installed keras-2.4.3
Python interpreter will be restarted.

In [0]:
import warnings
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import mlflow.keras
import mlflow.pyfunc

In [0]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
print(f"mlflow version={mlflow.__version__};keras version={keras.__version__};tensorlow version={tf.__version__}")

mlflow version=1.11.0;keras version=2.4.3;tensorlow version=2.3.0

[Source](https://androidkt.com/linear-regression-model-in-keras/)
Modified and extended for this tutorial

Problem: Build a simple Linear NN Model that predicts Celsius temperaturers from training data with Fahrenheit degree

<table>
  <tr><td>
    <img src="https://raw.githubusercontent.com/dmatrix/mlflow-workshop-part-2/master/images/temperature-conversion.png"
         alt="Bank Note " width="600">
  </td></tr>
</table>

Generate our X, y, and predict data

In [0]:
def f2c(f):
  return (f - 32) * 5.0/9.0

def gen_data(start, stop, step):
    X_fahrenheit = np.arange(start, stop, step, dtype=float)
    # Randomize the input
    np.random.shuffle(X_fahrenheit)
    y_celsius = np.array(np.array([f2c(f) for f in X_fahrenheit]))

    # generate Fahrenheit data to predict Celius 
    predict_data =[]
    [predict_data.append(t) for t in range (212, 170, -5)]
    
    return (X_fahrenheit, y_celsius, predict_data)

#### Define Inference Functions

In [0]:
def predict_keras_model(uri, data):
  model = mlflow.keras.load_model(uri)
  return [(f"(F={f}, C={model.predict([f])[0]})") for f in data]

def predict_pyfunc_model(uri, data):
  # Given Fahrenheit -> Predict Celcius
  # Create a pandas DataFrame with Fahrenheit unseen values
  # Get the Celius prediction
  pyfunc_model = mlflow.pyfunc.load_model(uri)
  df = pd.DataFrame(np.array(data))
  return pyfunc_model.predict(df)

#### Build a Keras Dense NN model

In [0]:
# Define the model
def baseline_model():
   model = keras.Sequential([
      keras.layers.Dense(64, activation='relu', input_shape=[1]),
      keras.layers.Dense(64, activation='relu'),
      keras.layers.Dense(1)
   ])

   optimizer = keras.optimizers.RMSprop(0.001)

   # Compile the model
   model.compile(loss='mean_squared_error',
                 optimizer=optimizer,
                 metrics=['mean_absolute_error', 'mean_squared_error'])
   return model

Capture run metrics using MLflow API

With `model.<flavor>.autolog()`, you don't need to log parameter, metrics, models etc.
autlogging is a convenient method to make it easier to use MLflow Fluent APIs, hence making your code eaiser to read.
  * Autologgin Features for [Model Flavors](https://mlflow.org/docs/latest/tracking.html#automatic-logging)
  * PR for in development:
    * `mlflow.pytorch.autolog()`
    * `mlflow.scklearn.autolog()`

In [0]:
def mlflow_run(params, X, y, run_name="Keras Linear Regression"):

   # Start MLflow run and log everyting...
   with mlflow.start_run(run_name=run_name) as run:
      
      run_id = run.info.run_uuid
      exp_id = run.info.experiment_id
      
      model = baseline_model()
      # single line of MLflow Fluent API obviates the need to log
      # individual parameters, metrics, model, artifacts etc...
      # https://mlflow.org/docs/latest/python_api/mlflow.keras.html#mlflow.keras.autolog
      mlflow.keras.autolog()
      model.fit(X, y, batch_size=params['batch_size'], epochs=params['epochs'])

      return (exp_id, run_id)

#### Train the Keras model

### Run 1

Generate X, y, and predict_data

In [0]:
(X, y, predict_data) = gen_data(-212, 10512, 2)

In [0]:
params = {'batch_size': 10,'epochs': 100}
(exp_id, run_id) = mlflow_run(params, X, y)
print(f"Finished Experiment id={exp_id} and run id = {run_id}")

Epoch 1/100
 1/537 [..............................] - ETA: 0s - loss: 7215835.0000 - mean_absolute_error: 2510.9336 - mean_squared_error: 7215835.0000 51/537 [=>............................] - ETA: 0s - loss: 481741.1562 - mean_absolute_error: 277.8282 - mean_squared_error: 481741.1562 101/537 [====>.........................] - ETA: 0s - loss: 246904.0469 - mean_absolute_error: 163.1308 - mean_squared_error: 246904.0469155/537 [=======>......................] - ETA: 0s - loss: 163120.2188 - mean_absolute_error: 122.6082 - mean_squared_error: 163120.2188209/537 [==========>...................] - ETA: 0s - loss: 122829.1953 - mean_absolute_error: 104.3409 - mean_squared_error: 122829.1953259/537 [=============>................] - ETA: 0s - loss: 100999.5703 - mean_absolute_error: 97.8080 - mean_squared_error: 100999.5703 311/537 [================>.............] - ETA: 0s - loss: 85275.5156 - mean_absolute_error: 89.2342 - mean_squared_error: 85275.5156 364/537 [===================>..........] - ETA: 0s - loss: 74024.7578 - mean_absolute_error: 84.4542 - mean_squared_error: 74024.7578418/537 [======================>.......] - ETA: 0s - loss: 65682.0938 - mean_absolute_error: 81.9340 - mean_squared_error: 65682.0938472/537 [=========================>....] - ETA: 0s - loss: 58994.5508 - mean_absolute_error: 77.8489 - mean_squared_error: 58994.5508525/537 [============================>.] - ETA: 0s - loss: 53752.2344 - mean_absolute_error: 75.1767 - mean_squared_error: 53752.2344537/537 [==============================] - 1s 1ms/step - loss: 52721.6172 - mean_absolute_error: 74.4843 - mean_squared_error: 52721.6172
Epoch 2/100
 1/537 [..............................] - ETA: 0s - loss: 16369.4482 - mean_absolute_error: 115.5505 - mean_squared_error: 16369.4482 59/537 [==>...........................] - ETA: 0s - loss: 6995.7974 - mean_absolute_error: 53.3445 - mean_squared_error: 6995.7974 112/537 [=====>........................] - ETA: 0s - loss: 7578.9009 - mean_absolute_error: 52.8887 - mean_squared_error: 7578.9009168/537 [========>.....................] - ETA: 0s - loss: 692

### Check the MLflow UI
 * check the Model file
 * check the Conda.yaml file
 * check the metrics

### Load the Keras Model Flavor: Native Flavor

In [0]:
# Load this Keras Model Flavor as a Keras native model flavor and make a prediction
model_uri = f"runs:/{run_id}/model"
print(f"Loading the Keras Model={model_uri} as Keras Model")
predictions = predict_keras_model(model_uri, predict_data)
print(predictions)

Loading the Keras Model=runs:/893e1496260e48b290b635e94a741f0b/model as Keras Model
['(F=212, C=[100.34866])', '(F=207, C=[97.56474])', '(F=202, C=[94.78084])', '(F=197, C=[91.99694])', '(F=192, C=[89.21303])', '(F=187, C=[86.42912])', '(F=182, C=[83.645226])', '(F=177, C=[80.86131])', '(F=172, C=[78.07741])']

### Load the Keras Model Flavor as a PyFunc Flavor

In [0]:
# Load this Keras Model Flavor as a pyfunc model flavor and make a prediction
pyfunc_uri = f"runs:/{run_id}/model"
print(f"Loading the Keras Model={pyfunc_uri} as Pyfunc Model")
predictions = predict_pyfunc_model(pyfunc_uri, predict_data)
print(predictions)

Loading the Keras Model=runs:/893e1496260e48b290b635e94a741f0b/model as Pyfunc Model
 0
0 100.348648
1 97.564728
2 94.780830
3 91.996933
4 89.213028
5 86.429115
6 83.645218
7 80.861320
8 78.077415

### Excercise/Lab Assignment

Using what we have learning in this session:
  * Can you decrease the MSE?
  * Increase the size of input data
  * Change the batch size and number of epochs
      * Run at least three experiments with different parameters: number of epochs, batches
  * Compare the runs and metrics
  
Were you able lower the MSE?

**CHALLENGE**: Excercise/Lab Assignment
 * Convert this program as a GitHub MLflow Project
 * Use the Lab 1 [MLflow Project](https://github.com/mlflow/mlflow-example) as a template
 * Other examples are in [here](https://github.com/mlflow/mlflow/tree/master/examples)